# Análisis del efecto de fumar en el peso de los recien nacidos

Puedes encontrar información del dataset [aquí](https://rpubs.com/phil1234/916282)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import statsmodels.api as sm

In [2]:
babies = pd.read_csv("data/ncbirths.csv")

In [4]:
babies.dropna(inplace=True)

In [6]:
babies

,fage,mage,mature,weeks,premie,visits,marital,gained,weight,lowbirthweight,gender,habit,whitemom
2,19.0,15,younger mom,37.0,full term,11.0,not married,38.0,6.63,not low,female,nonsmoker,white
3,21.0,15,younger mom,41.0,full term,6.0,not married,34.0,8.00,not low,male,nonsmoker,white
6,18.0,15,younger mom,37.0,full term,12.0,not married,76.0,8.44,not low,male,nonsmoker,not white
7,17.0,15,younger mom,35.0,premie,5.0,not married,15.0,4.69,low,male,nonsmoker,not white
9,20.0,16,younger mom,37.0,full term,13.0,not married,52.0,6.94,not low,female,nonsmoker,white
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,47.0,42,mature mom,40.0,full term,10.0,married,26.0,8.44,not low,male,nonsmoker,not white
996,34.0,42,mature mom,38.0,full term,18.0,married,20.0,6.19,not low,female,nonsmoker,white
997,39.0,45,mature mom,40.0,full term,15.0,married,32.0,6.94,not low,female,nonsmoker,white
998,55.0,46,mature mom,31.0,premie,8.0,married,25.0,4.56,low,female,nonsmoker,not white


In [11]:
babies["mature"] = babies["mature"].map({"younger mom": 1, "mature mom":0})
babies['premie'] = babies["premie"].map({"premie": 1, "full term":0})
babies['marital'] = babies["marital"].map({"not married": 1, "married":0})
babies['gender'] = babies["gender"].map({"female": 1, "male":0})
babies['habit'] = babies["habit"].map({"smoker": 1, "nonsmoker":0})
babies['whitemom'] = babies["whitemom"].map({"not white": 1, "white":0})

In [12]:
babies

,fage,mage,mature,weeks,premie,visits,marital,gained,weight,lowbirthweight,gender,habit,whitemom
2,19.0,15,1,37.0,0,11.0,1,38.0,6.63,not low,1,0,0
3,21.0,15,1,41.0,0,6.0,1,34.0,8.00,not low,0,0,0
6,18.0,15,1,37.0,0,12.0,1,76.0,8.44,not low,0,0,1
7,17.0,15,1,35.0,1,5.0,1,15.0,4.69,low,0,0,1
9,20.0,16,1,37.0,0,13.0,1,52.0,6.94,not low,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,47.0,42,0,40.0,0,10.0,0,26.0,8.44,not low,0,0,1
996,34.0,42,0,38.0,0,18.0,0,20.0,6.19,not low,1,0,0
997,39.0,45,0,40.0,0,15.0,0,32.0,6.94,not low,1,0,0
998,55.0,46,0,31.0,1,8.0,0,25.0,4.56,low,1,0,1


In [20]:
# Creación del modelo utilizando matrices como en scikitlearn
# ==============================================================================
# A la matriz de predictores se le tiene que añadir una columna de 1s para el intercept del modelo

# La convenciones determinan que las features se llamen X 
# Además, tendremos habitualmente un conjunto de train y otro de test (con un split del 80/20, habitualmente)
# Por lo tanto, X_train contiene las features del train
# Este caso es tan sencillo y con pocos datos que no tenemos X_test, pero aparecerá
X_train = babies[["mage", "weeks", "marital", "gained", "gender", "habit", "whitemom"]]

# Añadimos una columna de 1 porque necesitamos una constante que determine el intercept o B0, el punto donde corta la recta con el eje Y
# Por eso añadimos una columna constante de 0s
# Veremos que estoy aparece en muchos otros modelos (en redes neuronales se llama bias)
X_train = sm.add_constant(X_train, prepend=True)
# OLS: creamos el modelo del tipo Ordinary Least Squares (por alguna razón creo que una vez dije Optimal pero es Ordinary)
modelo = sm.OLS(endog=babies['weight'], exog=X_train)
# Ajustamos el modelo, es decir, hacemos el calculo de la mejor recta posible según los criterios del OLS
modelo = modelo.fit()
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:                 weight   R-squared:                       0.454
Model:                            OLS   Adj. R-squared:                  0.449
Method:                 Least Squares   F-statistic:                     94.15
Date:                Wed, 08 May 2024   Prob (F-statistic):          9.76e-100
Time:                        19:40:29   Log-Likelihood:                -1184.7
No. Observations:                 800   AIC:                             2385.
Df Residuals:                     792   BIC:                             2423.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -5.3200      0.592     -8.992      0.0

In [42]:
# Creación del modelo utilizando matrices como en scikitlearn
# ==============================================================================
# A la matriz de predictores se le tiene que añadir una columna de 1s para el intercept del modelo

# La convenciones determinan que las features se llamen X 
# Además, tendremos habitualmente un conjunto de train y otro de test (con un split del 80/20, habitualmente)
# Por lo tanto, X_train contiene las features del train
# Este caso es tan sencillo y con pocos datos que no tenemos X_test, pero aparecerá
X_train = babies[["mage", "weeks", "marital", "gained", "gender", "habit", "whitemom"]]
#X_train = babies.drop(columns=["weight", "lowbirthweight"])

# Añadimos una columna de 1 porque necesitamos una constante que determine el intercept o B0, el punto donde corta la recta con el eje Y
# Por eso añadimos una columna constante de 0s
# Veremos que estoy aparece en muchos otros modelos (en redes neuronales se llama bias)
X_train = sm.add_constant(X_train, prepend=True)
# OLS: creamos el modelo del tipo Ordinary Least Squares (por alguna razón creo que una vez dije Optimal pero es Ordinary)
modelo = sm.OLS(endog=babies["weight"], exog=X_train)
# Ajustamos el modelo, es decir, hacemos el calculo de la mejor recta posible según los criterios del OLS
modelo = modelo.fit()
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:                 weight   R-squared:                       0.454
Model:                            OLS   Adj. R-squared:                  0.449
Method:                 Least Squares   F-statistic:                     94.15
Date:                Thu, 09 May 2024   Prob (F-statistic):          9.76e-100
Time:                        09:43:02   Log-Likelihood:                -1184.7
No. Observations:                 800   AIC:                             2385.
Df Residuals:                     792   BIC:                             2423.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -5.3200      0.592     -8.992      0.0

In [43]:
# Creación del modelo utilizando matrices como en scikitlearn
# ==============================================================================
# A la matriz de predictores se le tiene que añadir una columna de 1s para el intercept del modelo

# La convenciones determinan que las features se llamen X 
# Además, tendremos habitualmente un conjunto de train y otro de test (con un split del 80/20, habitualmente)
# Por lo tanto, X_train contiene las features del train
# Este caso es tan sencillo y con pocos datos que no tenemos X_test, pero aparecerá
X_train = babies[["weeks", "marital", "gained", "gender", "habit", "whitemom"]]
#X_train = babies.drop(columns=["weight", "lowbirthweight"])

# Añadimos una columna de 1 porque necesitamos una constante que determine el intercept o B0, el punto donde corta la recta con el eje Y
# Por eso añadimos una columna constante de 0s
# Veremos que estoy aparece en muchos otros modelos (en redes neuronales se llama bias)
X_train = sm.add_constant(X_train, prepend=True)
# OLS: creamos el modelo del tipo Ordinary Least Squares (por alguna razón creo que una vez dije Optimal pero es Ordinary)
modelo = sm.OLS(endog=babies["weight"], exog=X_train)
# Ajustamos el modelo, es decir, hacemos el calculo de la mejor recta posible según los criterios del OLS
modelo = modelo.fit()
print(modelo.summary())

                            OLS Regression Results                            
Dep. Variable:                 weight   R-squared:                       0.453
Model:                            OLS   Adj. R-squared:                  0.449
Method:                 Least Squares   F-statistic:                     109.6
Date:                Thu, 09 May 2024   Prob (F-statistic):          1.69e-100
Time:                        09:43:02   Log-Likelihood:                -1185.4
No. Observations:                 800   AIC:                             2385.
Df Residuals:                     793   BIC:                             2418.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -5.0427      0.539     -9.357      0.0

In [44]:
modelo.resid.mean()

4.156675004196586e-15

In [45]:
X_train.corr()

,const,weeks,marital,gained,gender,habit,whitemom
const,NaN,NaN,NaN,NaN,NaN,NaN,NaN
weeks,NaN,1.000000,-0.057601,0.098580,-0.013934,-0.018180,-0.079184
marital,NaN,-0.057601,1.000000,0.014800,0.003420,0.051816,0.243026
gained,NaN,0.098580,0.014800,1.000000,-0.030932,0.010478,-0.062082
gender,NaN,-0.013934,0.003420,-0.030932,1.000000,-0.036905,-0.048532
habit,NaN,-0.018180,0.051816,0.010478,-0.036905,1.000000,-0.059430
whitemom,NaN,-0.079184,0.243026,-0.062082,-0.048532,-0.059430,1.000000


In [46]:
from sklearn.model_selection import train_test_split

# Esta vez, nuestro modelo serán el 80% de los datos y el test el 20% de los datos
X_train, X_test, y_train, y_test = train_test_split(babies[["weeks", "marital", "gained", "gender", "habit", "whitemom"]], babies["weight"], test_size = 0.20, random_state = 42)

In [47]:
# Creación del modelo utilizando matrices como en scikitlearn
# ==============================================================================
# A la matriz de predictores se le tiene que añadir una columna de 1s para el intercept del modelo

# La convenciones determinan que las features se llamen X 
# Además, tendremos habitualmente un conjunto de train y otro de test (con un split del 80/20, habitualmente)
# Por lo tanto, X_train contiene las features del train
# Este caso es tan sencillo y con pocos datos que no tenemos X_test, pero aparecerá
#X_train = babies[["weeks", "marital", "gained", "gender", "habit", "whitemom"]]
#X_train = babies.drop(columns=["weight", "lowbirthweight"])

# Añadimos una columna de 1 porque necesitamos una constante que determine el intercept o B0, el punto donde corta la recta con el eje Y
# Por eso añadimos una columna constante de 0s
# Veremos que estoy aparece en muchos otros modelos (en redes neuronales se llama bias)
X_train = sm.add_constant(X_train, prepend=True)
# OLS: creamos el modelo del tipo Ordinary Least Squares (por alguna razón creo que una vez dije Optimal pero es Ordinary)
modelo_ = sm.OLS(endog=y_train, exog=X_train)
# Ajustamos el modelo, es decir, hacemos el calculo de la mejor recta posible según los criterios del OLS
modelo_ = modelo_.fit()
print(modelo_.summary())

                            OLS Regression Results                            
Dep. Variable:                 weight   R-squared:                       0.452
Model:                            OLS   Adj. R-squared:                  0.447
Method:                 Least Squares   F-statistic:                     86.98
Date:                Thu, 09 May 2024   Prob (F-statistic):           2.32e-79
Time:                        09:43:02   Log-Likelihood:                -951.68
No. Observations:                 640   AIC:                             1917.
Df Residuals:                     633   BIC:                             1949.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -5.0334      0.612     -8.220      0.0

In [48]:
# Predice los precios para X_test usando este nuevo modelo y el anterior
# utiliza el método predict que tienen los modelos 
